Pipeline to get features on single text

In [1]:
import numpy as np
from tqdm import tqdm, trange
import glob

Для расчета дырок
1. файл filtered_by_*
2. файлы ru_word_holes_* в папке holes (!поменять индексы срезов в функции get_holes_per_chunk в зависимости от названий файлов)
3. файлы ru_word_* (наши чанки) в папке split_space (!поменять название файла в get_hole_embeddings)

Для извлечения фич
4. папка features (туда будут сохраняться файлы)
5. ru_cbow_normalized_dict (модель)

### some info

In [ ]:
by_syn_dang = np.load('filtered_by_lifetimes_and_syn.npy', allow_pickle=True).item()
by_syn_dang

{'0': array([488]), '1111': array([400, 415, 435, 444, 470, 492])}

In [ ]:
by_lt_dang = np.load('filtered_by_lifetimes (1).npy', allow_pickle=True).item()
by_lt_dang

{'0': np.int64(203),
 '10': np.int64(444),
 '110': np.int64(485),
 '1110': np.int64(493),
 '1111': np.int64(291),
 '1112': -1,
 '1113': np.int64(496),
 '112': np.int64(486),
 '113': np.int64(227),
 '114': np.int64(477),
 '12': np.int64(457),
 '13': np.int64(453),
 '14': np.int64(439),
 '2': np.int64(376),
 '3': np.int64(286),
 '4': np.int64(302),
 '5': np.int64(380),
 '6': np.int64(479),
 '7': np.int64(424)}

In [ ]:
by_syn = np.load('filtered_by_lifetimes_and_syn_norm_km5.npy', allow_pickle=True).item()
by_diam995 = np.load('filtered_by_lifetimes_and_hole_diam_norm_km5_995.npy', allow_pickle=True).item()
by_diam99 = np.load('filtered_by_lifetimes_and_hole_diam_norm_km5_99.npy', allow_pickle=True).item()
print(f'SYN: {by_syn}')
print(f'DIAM Q=.995: {by_diam995}')
print(f'DIAM Q=.99: {by_diam99}')

SYN: {'0': array([403, 437])}
DIAM Q=.995: {'0': array([403, 437, 477, 486])}
DIAM Q=.99: {'0': array([237, 375, 403, 426, 437, 470, 477, 486])}


BY SYN KM=5

Hole 403: ['проучивать', 'припугивать', 'прирезывать', 'прирезать', 'загрызать', 'укусить', 'цапнуть', 'вцепляться', 'сжимать', 'сжиматься', 'защемлять', 'щемить', 'щемящий', 'сосущий', 'глодать', 'снедать', 'пересиливать', 'побороть', 'совладать', 'утихомиривать', 'угомонять', 'приструнивать']

Hole 437: ['холодильник', 'пицца', 'пирожок', 'дожевывать', 'откусывать', 'отгрызать', 'цапнуть', 'отпихивать', 'отпрыгивать', 'рвануться', 'ринуться', 'помчаться', 'ускакивать', 'пришпоривать', 'седлать', 'расседлывать', 'мул', 'двуколка', 'пролетка', 'кабриолет', 'джип', 'гараж', 'подъезд', 'крыльцо', 'веранда', 'балкон', 'лоджия', 'кладовка']


In [ ]:
by_syn = np.load('filtered_by_lifetimes_and_syn_norm.npy', allow_pickle=True).item()
by_diam995 = np.load('filtered_by_lifetimes_and_hole_diam_norm.npy', allow_pickle=True).item()

print(f'SYN KM=10: {by_syn}')
print(f'DIAM KM=10 Q=.995: {by_diam995}')

SYN KM=10: {'3': array([389, 463])}
DIAM KM=10 Q=.995: {'3': array([281, 389, 403, 443, 463, 499])}


BY SYN KM=10

Hole 389: ['телохранитель', 'гвардеец', 'оруженосец', 'рыцарь', 'воин', 'орк', 'вожак', 'волк', 'волчонок', 'котенок', 'малыш', 'сестренка', 'дочурка', 'внучка', 'племянница', 'баронесса', 'цесаревна', 'михайлович', 'генерал', 'майор', 'комиссар', 'инспектор', 'охранник', 'констебль']

Hole 463: ['полиция', 'милиция', 'мент', 'бандит', 'отморозок', 'лох', 'говнюк', 'засранец', 'стерва', 'бабник', 'ловелас', 'хлыщ', 'аристократка', 'особа', 'леди', 'баронесса', 'цесаревна', 'михайлович', 'генерал', 'майор', 'комиссар', 'полицейский', 'инспектор']


## get hole embeddings

In [2]:
def get_cycles(cycles):
    """
        Get homology representatives (cycles) based on list of connected edges.
    """
    new_cycle_flag = True
    cycles_list = []
    cur_cycle = set()
    for a, b in cycles:
        if new_cycle_flag:
            cur_cycle.update([a])
            cur_cycle.update([b])
            cycles_list.append([])
            new_cycle_flag = False
            continue
        if a in cur_cycle:
            cur_cycle.remove(a)
            cycles_list[-1].append(a)
        else:
            cur_cycle.update([a])
        if b in cur_cycle:
            cur_cycle.remove(b)
            cycles_list[-1].append(b)
        else:
            cur_cycle.update([b])
        if len(cur_cycle) == 0:
            new_cycle_flag = True
    return cycles_list

In [3]:
def get_holes_per_chunk(dir="holes", filenames="ru_word_holes_*.npy"):
    """
        Get all homology representatives (cycles) for each chunk.
    """
    hole_contours = {}
    for file in glob.glob(f"{dir}/{filenames}"):
        hole_contours["_".join(file.split('_')[4:])[:-4]] = get_cycles(np.load(file) - 1) # 5 - позиция индекса чанка в названии файла ru_hord_holes_*
    return hole_contours

In [4]:
def get_hole_embeddings(lang='ru', part='word', filtered_path = "filtered_by_lifetimes_and_syn_norm_km5.npy"):
    """
    Dictionary keys: {chunk_number}${hole_number}: embeddings
    """

    filtered = np.load(filtered_path, allow_pickle=True).item()

    # получаем контуры дырок
    h1_contours = get_holes_per_chunk(dir='holes', filenames=f"{lang}_{part}_holes_*.npy")

    hole_embeddings = {}

    # итерируемся по чанкам в filtered
    for c in filtered:
        embs = np.load(f'split_space/ru_word_norm_{c}.npy') # вот тут менять если что
        for hole_num in filtered[c]:
            hole_embeddings[f"{c}${hole_num}"] = embs[h1_contours[c][hole_num]]

    filtered_name = '_'.join(filtered_path.split('_')[1:])
    np.save(f"holes/hole_embeddings_{filtered_name}", hole_embeddings)

In [ ]:
hole_diam_km5_995 = '/content/filtered_by_lifetimes_and_hole_diam_norm_km5_995.npy'
get_hole_embeddings(filtered_path=hole_diam_km5_995)

dict_keys(['4', '2', '3', '0', '1'])


In [ ]:
hole_diam_km5_99 = '/content/filtered_by_lifetimes_and_hole_diam_norm_km5_99.npy'
get_hole_embeddings(filtered_path=hole_diam_km5_99)

In [ ]:
hole_syn_km5 = '/content/filtered_by_lifetimes_and_syn_norm_km5.npy'
get_hole_embeddings(filtered_path=hole_syn_km5)

In [5]:
hole_syn_km10 = '/content/filtered_by_lifetimes_and_syn_norm.npy'
get_hole_embeddings(filtered_path=hole_syn_km10)

## funcs

In [6]:
def get_word_space(model, text):
    corpus = text
    words = set([w for w in corpus.split() if w in model])
    space = np.vstack([model[w] for w in words])
    return space


from sklearn.metrics import pairwise_distances
def get_dist_to_centers_array(e, hole_e_centers):
    a = pairwise_distances(e, hole_e_centers, metric='cosine')
    return np.hstack([a, a.mean(axis=1).reshape(-1, 1)])

from collections import Counter
def get_dist_array(e, hole_embs, apply_func=np.min):
    dist_list = np.vstack([apply_func(pairwise_distances(e, hole, metric='cosine'), axis=1) for hole in hole_embs])
    dist_list = np.vstack([dist_list, dist_list.mean(axis=0)])
    return dist_list.T
def get_most_common_closest_hole(min_dist):
    cnt = Counter(min_dist.argmin(axis=1))
    a = np.array([cnt[hn] for hn in range(min_dist.shape[1])]) / min_dist.shape[0]
    return np.hstack([a, [a.argmax()]])

In [7]:
import pandas as pd
'''
 model - наш словарь эмбеддингов
 texts - список строк из файла
 holes_type - путь к файлу с соответсвующими эмбеддингами дырок (по синонимам/ по диаметру)
'''
def process(texts, model,
            lang='RU',
            holes_path='/content/holes/hole_embeddings_by_lifetimes_and_syn_norm_km5.npy',
            text_type='lit', hole_type='syn'):

    features = []
    text_names = []
    # for part in ['word', 'bigram', 'trigram']:
    hole_embeddings = np.load(holes_path, allow_pickle=True).item() # загружаем эмбеддинги дырок

    hole_e_centers = np.vstack([h.mean(axis=0) for h in hole_embeddings.values()]) # получаем центры

    for i, f in tqdm(enumerate(texts), desc="Processing words..."):

      try:
        space = get_word_space(model, f) # слова текста

        c_mean = np.mean(get_dist_to_centers_array(space, hole_e_centers), axis=0)
        m1 = get_dist_array(space, hole_embeddings.values(), np.min)
        m2_mean = np.mean(get_dist_array(space, hole_embeddings.values(), np.max), axis=0)
        h = get_most_common_closest_hole(m1[:,:-1])
        features.append(np.hstack([c_mean, np.mean(m1, axis=0), m2_mean, h]))

        name = f'{text_type}_{i}'
        text_names.append(name)
      except:
        continue

    data = pd.DataFrame(features)
    data['text'] = text_names
    data.to_csv(f"features/{lang}_{text_type}_{hole_type}_features.csv", index=False)
    print(f"Saved at features/{lang}_{text_type}_{hole_type}_features.csv")

## extracting

In [10]:
ru_model = np.load("ru_cbow_normalized_dict.npy", allow_pickle=True).item()

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### models

In [16]:
text_type = 'bigmGPT'
lang = 'RU'
hole_emb = '/content/holes/hole_embeddings_by_lifetimes_and_syn_norm.npy'
hole_type = 'syn10'

with open('/content/drive/MyDrive/classifier/russian_bigmGPT_corpus.txt', encoding='utf-8') as f:
  texts = f.readlines()

process(texts, ru_model, lang=lang, holes_path=hole_emb, text_type=text_type, hole_type=hole_type)

Processing words...: 501it [00:08, 58.26it/s]

Saved at features/RU_bigmGPT_syn10_features.csv


In [ ]:
check = pd.read_csv('/content/features/RU_bigmGPT_word_features.csv')
check.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,text
0,1.082225,1.089637,1.013094,1.036894,1.055462,0.866977,0.839177,0.814157,0.835120,0.838858,...,1.229846,1.176558,1.186171,1.198987,0.221277,0.263830,0.255319,0.259574,1.0,bigmGPT_0
1,1.076907,1.078530,1.017054,1.028771,1.050315,0.865069,0.828327,0.821411,0.834726,0.837383,...,1.223945,1.173936,1.179400,1.194596,0.216741,0.288490,0.251121,0.243647,1.0,bigmGPT_1
2,1.072706,1.064612,1.002672,1.012025,1.038004,0.859922,0.813129,0.805533,0.816986,0.823893,...,1.225792,1.170201,1.174063,1.193014,0.197640,0.286136,0.241888,0.274336,1.0,bigmGPT_2
3,1.062818,1.059684,1.019625,1.022787,1.041227,0.852759,0.813511,0.817508,0.825116,0.827224,...,1.219540,1.180324,1.181757,1.194942,0.222558,0.293555,0.239174,0.244713,1.0,bigmGPT_3
4,1.090381,1.130216,1.018758,1.048497,1.071963,0.877886,0.863891,0.821291,0.851104,0.853543,...,1.243042,1.170997,1.188387,1.202265,0.242268,0.201031,0.319588,0.237113,2.0,bigmGPT_4


In [14]:
text_type = 'bigbalaboba'
lang = 'RU'
hole_emb = '/content/holes/hole_embeddings_by_lifetimes_and_syn_norm.npy'
hole_type = 'syn10'

with open('/content/drive/MyDrive/classifier/russian_bigbalaboba_corpus.txt', encoding='utf-8') as f:
  texts = f.readlines()

print(len(texts))
process(texts, ru_model, lang=lang, holes_path=hole_emb, text_type=text_type, hole_type=hole_type)

1039


Processing words...: 1039it [00:28, 36.52it/s]

Saved at features/RU_bigbalaboba_syn10_features.csv


In [ ]:
check = pd.read_csv('/content/features/RU_bigbalaboba_word_features.csv')
check.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,text
0,1.037670,1.081484,1.000895,1.034669,1.038680,0.841711,0.835470,0.816527,0.831690,0.831350,...,1.219895,1.157952,1.180791,1.184832,0.258537,0.248780,0.246341,0.246341,0.0,bigbalaboba_0
1,1.022798,1.051566,0.995250,1.011317,1.020233,0.825000,0.816158,0.814067,0.821650,0.819219,...,1.210596,1.158814,1.172144,1.180965,0.299229,0.246671,0.227050,0.227050,0.0,bigbalaboba_1
2,1.026721,1.066889,0.999182,1.024820,1.029404,0.836126,0.820326,0.816244,0.830463,0.825790,...,1.217098,1.155497,1.177516,1.182086,0.274468,0.257447,0.261702,0.206383,0.0,bigbalaboba_2
3,1.018794,1.036963,1.005106,1.004517,1.016346,0.820147,0.799098,0.813539,0.811154,0.810985,...,1.209251,1.170011,1.173177,1.183088,0.280868,0.271370,0.211443,0.236318,0.0,bigbalaboba_3
4,0.994863,1.080708,1.006653,0.997705,1.019982,0.806740,0.852872,0.828562,0.808737,0.824228,...,1.209706,1.153820,1.164620,1.174054,0.324324,0.162162,0.209459,0.304054,0.0,bigbalaboba_4


In [13]:
text_type = 'biggpt2'
lang = 'RU'
hole_emb = '/content/holes/hole_embeddings_by_lifetimes_and_syn_norm.npy'
hole_type = 'syn10'

with open('/content/drive/MyDrive/classifier/russian_biggpt2_corpus.txt', encoding='utf-8') as f:
  texts = f.readlines()

print(len(texts))
process(texts, ru_model, lang=lang, holes_path=hole_emb, text_type=text_type, hole_type=hole_type)

2662


Processing words...: 2662it [00:35, 75.95it/s]


Saved at features/RU_biggpt2_syn10_features.csv


In [12]:
text_type = 'newlstm'
lang = 'RU'
hole_emb = '/content/holes/hole_embeddings_by_lifetimes_and_syn_norm.npy'
hole_type = 'syn10'

with open('/content/drive/MyDrive/classifier/russian_newlstm_corpus.txt', encoding='utf-8') as f:
  texts = f.readlines()

print(len(texts))
process(texts, ru_model, lang=lang, holes_path=hole_emb, text_type=text_type, hole_type=hole_type)

994


Processing words...: 994it [00:17, 55.29it/s]

Saved at features/RU_newlstm_syn10_features.csv


### lit

In [11]:
text_type = 'lit'
lang = 'RU'
hole_emb = '/content/holes/hole_embeddings_by_lifetimes_and_syn_norm.npy'
hole_type = 'syn10'

with open('/content/drive/MyDrive/classifier/russian_newlit_corpus.txt', encoding='utf-8') as f:
  texts = f.readlines()

print(len(texts))
process(texts, ru_model, lang=lang, holes_path=hole_emb, text_type=text_type, hole_type=hole_type)

6429


Processing words...: 6429it [03:44, 28.66it/s]


Saved at features/RU_lit_syn10_features.csv
